In [1]:
import pandas as pd
import glob

In [2]:
# Carica il CSV in un DataFrame
df = pd.read_csv('dataset/2012.csv')

In [77]:
#CREA UN DF CON TUTTI I CSV

# Specifica il percorso alla cartella che contiene i file CSV
percorso = 'dataset/*.csv'

# Usa glob per trovare tutti i file CSV nella cartella
file_csv = glob.glob(percorso)

# Leggi e unisci tutti i file in un unico DataFrame
df = pd.concat([pd.read_csv(file) for file in file_csv], ignore_index=True)

In [21]:
df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2012-01-01,MQ,4041,BMI,ORD,1715.0,1702.0,-13.0,9.0,1711.0,...,55.0,55.0,37.0,116.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-01-01,MQ,4041,ORD,BMI,1545.0,1540.0,-5.0,11.0,1551.0,...,50.0,39.0,23.0,116.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-01-01,MQ,4042,SGF,ORD,715.0,705.0,-10.0,12.0,717.0,...,115.0,105.0,75.0,438.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-01-01,MQ,4045,ORD,DBQ,1535.0,1529.0,-6.0,13.0,1542.0,...,55.0,46.0,31.0,147.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-01-01,MQ,4047,ORD,RIC,1600.0,1615.0,15.0,12.0,1627.0,...,115.0,109.0,86.0,642.0,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
print(df.columns)
print(len(df.columns))

Index(['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 27'],
      dtype='object')
28


# ETL

### Definizione chiavi

In [3]:
# Conta e stampa il numero di duplicati
num_duplicati = df.duplicated().sum()
print(f"Il dataset contiene {num_duplicati} duplicati.")

Il dataset contiene 0 duplicati.


In [3]:
# Oppure, per modificare il DataFrame originale direttamente, puoi usare inplace=True
df.drop_duplicates(inplace=True)

# Controlla se ci sono ancora duplicati
if not df.duplicated().any():
    print("Tutti i duplicati sono stati rimossi.")

Tutti i duplicati sono stati rimossi.


In [32]:
print(f"Il dataset contiene {df.size} elementi")

Il dataset contiene 170709336 elementi


In [50]:
candidate_columns = [
    'FL_DATE',
    'OP_CARRIER',
    'OP_CARRIER_FL_NUM',
    'ORIGIN',
    'DEST'
]

if not df[candidate_columns].duplicated().any():
    print(f"La colonna '{candidate_columns}' potrebbe essere una chiave primaria.")
else:
    print(f"La colonna '{candidate_columns}' contiene duplicati e non può essere una chiave primaria.")

La colonna '['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST']' potrebbe essere una chiave primaria.


In [8]:
df['OP_CARRIER_FL_NUM'].value_counts()

OP_CARRIER_FL_NUM
12      4030
16      4009
24      3877
55      3872
407     3847
        ... 
7314       1
9221       1
7334       1
7336       1
7442       1
Name: count, Length: 6670, dtype: int64

In [10]:
df[df['OP_CARRIER_FL_NUM'] == 12]['OP_CARRIER'].value_counts()

OP_CARRIER
WN    963
US    717
UA    503
HA    366
AS    366
VX    365
B6    261
FL    208
AA    155
DL    126
Name: count, dtype: int64

In [7]:
df[(df['OP_CARRIER_FL_NUM'] == 12) & (df['OP_CARRIER'] == 'WN')][['ORIGIN', 'DEST']].value_counts()

ORIGIN  DEST
HOU     DAL     284
DAL     ELP      76
        MAF      60
TPA     HOU      59
PHX     SAN      46
SAN     SMF      46
ELP     PHX      46
DAL     LBB      35
JAN     HOU      30
STL     MSP      26
DAL     STL      26
        OKC      25
OKC     MDW      25
MCO     HOU      24
BNA     HOU      24
DAL     TUL      21
TUL     PHX      20
PHX     RNO      20
RNO     SEA      20
HRL     HOU       9
MCI     LAX       4
LAX     SJC       4
DAL     MCI       4
MDW     MCI       2
LAS     MCO       1
PDX     RNO       1
BNA     DEN       1
TPA     BWI       1
BWI     BNA       1
SAT     HOU       1
DAL     BHM       1
        LIT       1
RNO     LAX       1
DEN     OAK       1
PHX     LAX       1
        ABQ       1
PHL     BNA       1
DEN     PHX       1
LAX     OAK       1
MKE     PHX       1
MDW     PHL       1
        LGA       1
DEN     SEA       1
MCO     CMH       1
HOU     RDU       1
MCI     DEN       1
MAF     DAL       1
LIT     MDW       1
ATL     HOU       1
LAX    

In [5]:
df_filtrato = df[(df['OP_CARRIER_FL_NUM'] == 12) & 
                 (df['OP_CARRIER'] == 'US')]

df_filtrato

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
3284,2012-01-01,US,12,PHX,JFK,956.0,955.0,-1.0,29.0,1024.0,...,296.0,289.0,254.0,2153.0,NaN,NaN,NaN,NaN,NaN,NaN
3285,2012-01-01,US,12,SMF,PHX,600.0,626.0,26.0,17.0,643.0,...,108.0,106.0,85.0,647.0,24.0,0.0,0.0,0.0,0.0,NaN
30748,2012-01-02,US,12,PHX,JFK,956.0,953.0,-3.0,27.0,1020.0,...,296.0,318.0,278.0,2153.0,0.0,0.0,19.0,0.0,0.0,NaN
30749,2012-01-02,US,12,SMF,PHX,600.0,556.0,-4.0,9.0,605.0,...,108.0,104.0,91.0,647.0,NaN,NaN,NaN,NaN,NaN,NaN
35178,2012-01-03,US,12,PHX,JFK,956.0,954.0,-2.0,24.0,1018.0,...,296.0,300.0,270.0,2153.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6063046,2012-12-29,US,12,SEA,PHX,515.0,512.0,-3.0,16.0,528.0,...,165.0,175.0,155.0,1107.0,NaN,NaN,NaN,NaN,NaN,NaN
6075501,2012-12-30,US,12,PHX,JFK,955.0,955.0,0.0,17.0,1012.0,...,291.0,252.0,228.0,2153.0,NaN,NaN,NaN,NaN,NaN,NaN
6075502,2012-12-30,US,12,SEA,PHX,515.0,508.0,-7.0,18.0,526.0,...,165.0,160.0,138.0,1107.0,NaN,NaN,NaN,NaN,NaN,NaN
6093317,2012-12-31,US,12,PHX,JFK,955.0,957.0,2.0,25.0,1022.0,...,291.0,249.0,215.0,2153.0,NaN,NaN,NaN,NaN,NaN,NaN


## Analisi NaN

In [51]:
# Per ogni colonna  calcola la percentuale di NaN e non NaN
for col in df.columns:
    nan_percent = df[col].isna().mean() * 100  # Percentuale di NaN
    non_nan_percent = 100 - nan_percent  # Percentuale di non NaN
    print(f"Colonna: {col}")
    print(f"Percentuale di NaN: {nan_percent:.2f}%")
    print(f"Percentuale di non NaN: {non_nan_percent:.2f}%")
    print("-" * 40)

Colonna: FL_DATE
Percentuale di NaN: 0.00%
Percentuale di non NaN: 100.00%
----------------------------------------
Colonna: OP_CARRIER
Percentuale di NaN: 0.00%
Percentuale di non NaN: 100.00%
----------------------------------------
Colonna: OP_CARRIER_FL_NUM
Percentuale di NaN: 0.00%
Percentuale di non NaN: 100.00%
----------------------------------------
Colonna: ORIGIN
Percentuale di NaN: 0.00%
Percentuale di non NaN: 100.00%
----------------------------------------
Colonna: DEST
Percentuale di NaN: 0.00%
Percentuale di non NaN: 100.00%
----------------------------------------
Colonna: CRS_DEP_TIME
Percentuale di NaN: 0.00%
Percentuale di non NaN: 100.00%
----------------------------------------
Colonna: DEP_TIME
Percentuale di NaN: 1.57%
Percentuale di non NaN: 98.43%
----------------------------------------
Colonna: DEP_DELAY
Percentuale di NaN: 1.57%
Percentuale di non NaN: 98.43%
----------------------------------------
Colonna: TAXI_OUT
Percentuale di NaN: 1.58%
Percentuale d

In [47]:
# Calcola la percentuale di NaN per ciascuna colonna
soglia_nan = 0.5  # soglia del 50% di valori NaN
colonne_more_05_nan = df.columns[df.isna().mean() > soglia_nan]

# Per ogni colonna che ha più del 50% di NaN, calcola la percentuale di NaN e non NaN
for col in colonne_more_05_nan:
    nan_percent = df[col].isna().mean() * 100  # Percentuale di NaN
    non_nan_percent = 100 - nan_percent  # Percentuale di non NaN
    print(f"Colonna: {col}")
    print(f"Percentuale di NaN: {nan_percent:.2f}%")
    print(f"Percentuale di non NaN: {non_nan_percent:.2f}%")
    print("-" * 40)

Colonna: CANCELLATION_CODE
Percentuale di NaN: 98.41%
Percentuale di non NaN: 1.59%
----------------------------------------


### Analisi delay

In [26]:
# Lista delle colonne *_DELAY che ti interessano (quelle nel tuo output)
delay_columns = [
    'CARRIER_DELAY',
    'WEATHER_DELAY',
    'NAS_DELAY',
    'SECURITY_DELAY',
    'LATE_AIRCRAFT_DELAY'
]

# Ci sono dei valori di Arr_Delay che hanno i valori di delay specifici settati a NAN
# Filtra le righe che hanno almeno un valore x_delay non NaN
delay_rows_non_nan = df.dropna(subset=delay_columns)

# Verifica se ci sono righe dove almeno un *_DELAY è non NaN e altre colonne sono NaN
mask_non_nan = delay_rows_non_nan[delay_columns].notna().all(axis=1)  # Righe con tutti i *_DELAY non NaN

print("Se un delay specifico è non nan allora tutti gli altri pure non lo saranno:" + str(mask_non_nan.all()))

# Trova le righe che non soddisfano il criterio
inconsistent_rows = delay_rows_non_nan[~mask_non_nan]

# Stampa le righe con inconsistenze
if not inconsistent_rows.empty:
    # nel caso un tipo di delay specifico è NaN anche tutti gli altri tipi specifici devono essere NaN
    for index in inconsistent_rows.index:
        print(f"Riga {index} ha NaN in una colonna *_DELAY mentre altre sono non NaN.")
else:
    # Alcuni valori di ARR_DELAY sono dati dalla somma degli altri valori delay
    condition = (delay_rows_non_nan[delay_columns].sum(axis=1) == delay_rows_non_nan['ARR_DELAY'])
    df_invalid = delay_rows_non_nan[~condition]

    if df_invalid.empty:
        print("Non ci sono inconsistenze nelle colonne di DELAY con i valori di DELAY specifi non NaN.")
        print("I valori di ARR_DELAY sono dati dalla somma degli altri valori delay specifi nel caso questi fossero non NaN")
    else:
        print("Ci sono inconsistenze")
        print(df_invalid[['ARR_DELAY'] + delay_columns])



delay_rows_non_nan[['ARR_DELAY'] + delay_columns ]


Se un delay specifico è non nan allora tutti gli altri pure non lo saranno:True
Non ci sono inconsistenze nelle colonne di DELAY con i valori di DELAY specifi non NaN.
I valori di ARR_DELAY sono dati dalla somma degli altri valori delay specifi nel caso questi fossero non NaN


,ARR_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
6,26.0,0.0,0.0,26.0,0.0,0.0
7,27.0,0.0,0.0,27.0,0.0,0.0
9,51.0,51.0,0.0,0.0,0.0,0.0
10,28.0,0.0,0.0,28.0,0.0,0.0
11,64.0,0.0,0.0,64.0,0.0,0.0
...,...,...,...,...,...,...
6096750,16.0,0.0,0.0,16.0,0.0,0.0
6096752,36.0,0.0,0.0,11.0,0.0,25.0
6096755,20.0,0.0,0.0,5.0,0.0,15.0
6096757,20.0,0.0,0.0,10.0,0.0,10.0


In [27]:
delay_rows_non_nan['OP_CARRIER'].value_counts()

OP_CARRIER
WN    180969
EV    154116
UA    111579
AA    109752
OO    101231
DL     92829
MQ     75466
US     52595
B6     44965
FL     26235
YV     18023
F9     17122
AS     16779
VX      8699
HA      4798
Name: count, dtype: int64

In [32]:
delay_rows_with_nan = df[df[delay_columns].isna().any(axis=1)]

# Ci sono riche che hanno ARR
if delay_rows_with_nan[delay_columns].isna().all(axis=1).all():
    # Verifica se ci sono sia valori positivi che negativi in ARR_DELAY
    positive_arr_delay = (delay_rows_with_nan['ARR_DELAY'] > 0).any()
    negative_arr_delay = (delay_rows_with_nan['ARR_DELAY'] < 0).any()

    if positive_arr_delay and negative_arr_delay:
        # ci sono righe che hanno tutti nan in delay specifici ed hanno sia arr_delay positivi che positivi
        print("Ci sono righe che hanno tutti nan in delay specifici ed hanno sia arr_delay negativi che positivi")


Ci sono righe che hanno tutti nan in delay specifici ed hanno sia arr_delay negativi che positivi


In [44]:
# Filtra ed elimina le righe con dep_delay > 0 e valori NaN nelle colonne specificate
df = df[~((df['DEP_DELAY'] > 0) & 
           (df[['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].isna().any(axis=1)))]

# Imposta a zero i valori delle colonne specificate se dep_delay <= 0
cols_to_zero = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
df.loc[df['DEP_DELAY'] <= 0, cols_to_zero] = 0

print(f"Il dataset contiene {df.size} elementi")

Il dataset contiene 135435776 elementi


In [46]:
# Stampa e elimina le colonne
df = df.drop(columns='Unnamed: 27')

### Analisi su cancelled e diverted

In [33]:
# Calcola la percentuale di 0 e 1 per le colonne "diverted" e "canceled"
percentuali = df[['DIVERTED', 'CANCELLED']].apply(lambda x: x.value_counts(normalize=True) * 100)
print(percentuali)

      DIVERTED  CANCELLED
0.0  99.794661  98.706494
1.0   0.205339   1.293506


In [35]:
# Filtra le righe dove "CANCELLED" è 1 e visualizza solo la colonna "CANCELLATION_CODE"
righe_canceled_1 = df.loc[df['CANCELLED'] == 1, 'CANCELLATION_CODE']


# Verifica se ci sono valori NaN nella colonna
if righe_canceled_1.isna().any():
    print("\nCi sono valori NaN nella colonna 'CANCELLATION_CODE' nelle righe dei voli cancellati, quindi che hanno CANCELED pari a 1.0")
else:
    print("\nTutti i valori nella colonna 'CANCELLATION_CODE' sono non NaN nelle righe dei voli cancellati, quindi che hanno CANCELED pari a 1.0.")

print("\nRighe con canceled = 1:")
righe_canceled_1

# Visualizza la frequenza di ciascun valore nella colonna "CANCELLATION_CODE"
print("\nValori unici nella colonna 'CANCELLATION_CODE' con la loro frequenza:")
righe_canceled_1.value_counts()

# A	Carrier	
# B	Weather	
# C	National Air System	
# D	Security


Tutti i valori nella colonna 'CANCELLATION_CODE' sono non NaN nelle righe dei voli cancellati, quindi che hanno CANCELED pari a 1.0.

Righe con canceled = 1:

Valori unici nella colonna 'CANCELLATION_CODE' con la loro frequenza:


CANCELLATION_CODE
B    33611
A    30291
C    14939
D       21
Name: count, dtype: int64

In [67]:
# SOSTITUIRE CANCELLATION_CODE CON CANCELLATION_REASON

# Mappa dei codici ai rispettivi motivi di cancellazione
cancellation_reasons = {
    'A': 'carrier',
    'B': 'weather',
    'C': 'national air system',
    'D': 'security'
}

# Trova l'indice della colonna CANCELLATION_CODE
col_index = df.columns.get_loc('CANCELLATION_CODE')

# Inserisce la nuova colonna CANCELLATION_REASON nello stesso punto
df.insert(col_index, 'CANCELLATION_REASON', df['CANCELLATION_CODE'].map(cancellation_reasons))

# Rimozione della colonna originale CANCELLATION_CODE
df = df.drop(columns=['CANCELLATION_CODE'])

            FL_DATE OP_CARRIER  OP_CARRIER_FL_NUM ORIGIN DEST  CRS_DEP_TIME  \
0        2009-01-01         XE               1204    DCA  EWR          1100   
1        2009-01-01         XE               1206    EWR  IAD          1510   
2        2009-01-01         XE               1207    EWR  DCA          1100   
3        2009-01-01         XE               1208    DCA  EWR          1240   
4        2009-01-01         XE               1209    IAD  EWR          1715   
...             ...        ...                ...    ...  ...           ...   
6429333  2009-12-31         YV               2846    YUM  PHX           615   
6429334  2009-12-31         YV               2850    YUM  PHX          1929   
6429335  2009-12-31         YV               2866    YUM  PHX          1400   
6429336  2009-12-31         YV               2868    YUM  PHX          1620   
6429337  2009-12-31         YV               2871    YUM  PHX          1123   

         DEP_TIME  DEP_DELAY  TAXI_OUT  WHEELS_OFF 

In [31]:
righe_canceled_2 = df.loc[df['CANCELLED'] == 0, 'CANCELLATION_CODE']

# Verifica se tutti i valori sono NaN
if righe_canceled_2.isna().all():
    print("\nTutti i valori nella colonna 'CANCELLATION_CODE' sono NaN nelle righe dei voli NON cancellati, quindi che hanno CANCELED pari a 0.0.")
else:
    print("\nNon tutti i valori nella colonna 'CANCELLATION_CODE' sono NaN nelle righe dei voli NON cancellati, quindi che hanno CANCELED pari a 0.0.")

print("\nRighe con canceled = 2:")
righe_canceled_2


Tutti i valori nella colonna 'CANCELLATION_CODE' sono NaN nelle righe dei voli NON cancellati.

Righe con canceled = 2:


0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
6429333    NaN
6429334    NaN
6429335    NaN
6429336    NaN
6429337    NaN
Name: CANCELLATION_CODE, Length: 6342300, dtype: object

In [41]:
df[df['CANCELLED'] == 1]


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
149,2012-01-01,MQ,4323,JFK,ORF,1700.0,NaN,NaN,NaN,NaN,...,95.0,NaN,NaN,290.0,NaN,NaN,NaN,NaN,NaN,NaN
390,2012-01-01,OO,6451,LAX,SMX,2235.0,NaN,NaN,NaN,NaN,...,59.0,NaN,NaN,134.0,NaN,NaN,NaN,NaN,NaN,NaN
405,2012-01-01,OO,5657,LAX,SBA,2245.0,NaN,NaN,NaN,NaN,...,47.0,NaN,NaN,89.0,NaN,NaN,NaN,NaN,NaN,NaN
475,2012-01-01,OO,4568,SAN,LAX,615.0,NaN,NaN,NaN,NaN,...,55.0,NaN,NaN,109.0,NaN,NaN,NaN,NaN,NaN,NaN
487,2012-01-01,OO,5644,SFO,PSC,2120.0,NaN,NaN,NaN,NaN,...,112.0,NaN,NaN,620.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6092772,2012-12-31,OO,6490,SFO,PSP,1246.0,NaN,NaN,NaN,NaN,...,88.0,NaN,NaN,421.0,NaN,NaN,NaN,NaN,NaN,NaN
6094204,2012-12-31,WN,117,BOS,STL,1125.0,NaN,NaN,NaN,NaN,...,195.0,NaN,NaN,1047.0,NaN,NaN,NaN,NaN,NaN,NaN
6096586,2012-12-31,YV,2763,PHX,TUS,1445.0,NaN,NaN,NaN,NaN,...,45.0,NaN,NaN,110.0,NaN,NaN,NaN,NaN,NaN,NaN
6096621,2012-12-31,YV,2703,TUS,PHX,1610.0,NaN,NaN,NaN,NaN,...,48.0,NaN,NaN,110.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Filtra le righe dove "diverted" è 1
righe_diverted_1 = df[df['DIVERTED'] == 1]

# Stampa o visualizza i risultati
print("Righe con diverted = 1:")
righe_diverted_1.head()

Righe con diverted = 1:


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
661,2009-01-01,XE,2980,IAH,TYS,1315,1312.0,-3.0,8.0,1320.0,...,122.0,NaN,NaN,772.0,NaN,NaN,NaN,NaN,NaN,NaN
818,2009-01-01,YV,2637,CLT,BHM,2230,2315.0,45.0,8.0,2323.0,...,85.0,NaN,NaN,351.0,NaN,NaN,NaN,NaN,NaN,NaN
1936,2009-01-01,OO,4490,MCI,ATL,1520,1523.0,3.0,11.0,1534.0,...,125.0,NaN,NaN,692.0,NaN,NaN,NaN,NaN,NaN,NaN
1994,2009-01-01,OO,4665,SLC,ACV,1115,1112.0,-3.0,16.0,1128.0,...,115.0,NaN,NaN,635.0,NaN,NaN,NaN,NaN,NaN,NaN
2192,2009-01-01,OO,4878,SLC,BTM,1059,1053.0,-6.0,15.0,1108.0,...,81.0,NaN,NaN,357.0,NaN,NaN,NaN,NaN,NaN,NaN
